# IBM Data Science Professional Certificate Capstone Projects

This notebook will be mainly used for capstone projects. This project is the final requirement to get the <b><i>IBM Data Science Professional Certificate<I></b>. 

This professional certificate was created for Coursera in colaboration with IBM and include a serie of eight courses plus a capstone project

In [2]:
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")


Hello Capstone Project Course!
